##  Stacking Model ##

In [1]:
# imports
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
from sklearn.metrics import accuracy_score
import random

In [2]:
class stacking_dataloader(Dataset):
    def __init__(self, path , train= True):
        self.path = path
        self.all_classes = ['NotSnow','Snow']
        self.class_indexes = {cls: idx for idx, cls in enumerate(self.all_classes)}
        self.dataset = self.data_lister(train)
        self.length = len(self.dataset)
        self.train_test = train
    def data_lister(self, train):
        data_list = []
        for class_name in self.all_classes:
            all_class_folders =os.path.join(self.path, class_name)
            for fold in os.listdir(all_class_folders):
                if fold == ".DS_Store":
                    continue
                sample_path_n_class = os.path.join(all_class_folders, fold)
                sample = (sample_path_n_class, self.class_indexes[class_name])
                data_list.append(sample)
        random.shuffle(data_list)
        
        if train:
            return data_list[:int(0.8*len(data_list))]
        else:
            return data_list[int(0.8*len(data_list)):]
    
    
    def __getitem__(self, index):
        sample_path, sample_class = self.dataset[index]
        pic1 = transforms.ToTensor()(Image.open(os.path.join(sample_path, '0.png')).convert('L')) # image 1 in tensor
        pic2 = transforms.ToTensor()(Image.open(os.path.join(sample_path, '1.png')).convert('L')) # image 2 in tensor
        sample = torch.cat((pic1, pic2), dim=0).unsqueeze(0)
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
        
        ])
        sample = transform(sample)
        return sample , sample_class

In [3]:
stacking_training = stacking_dataloader('./dataset/', True)
stacking_testing = stacking_dataloader('./dataset/', False)

In [4]:
print(f'the object returned is a tuple containing the tensor and the class label')
print(f'the shape of each tensor in this case is {stacking_training[0][0].shape}')

the object returned is a tuple containing the tensor and the class label
the shape of each tensor in this case is torch.Size([1, 2, 224, 224])


In [5]:
class stacking_CNN(nn.Module):
    def __init__(self):
        super(stacking_CNN, self).__init__()
        self.conv1  = nn.Conv2d(2,4, 2,stride= 1)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2 = nn.Conv2d(4, 5, 2, padding=1)
        self.conv3 = nn.Conv2d(5, 6, 3, padding=1)
        self.conv4 = nn.Conv2d(6, 7, 2, padding=1)
        self.conv5 = nn.Conv2d(7, 4, 3, padding=1)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(4, 2)
        
    def forward(self,x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv3(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv4(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.conv5(x)
        x = self.relu(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
        
    

In [6]:
learning_rate = 0.001
epochs = 10

In [7]:
model = stacking_CNN()
criterion = nn.BCEWithLogitsLoss()
optimiser = optim.Adam(model.parameters(), lr = learning_rate)


In [8]:
# model params
total_params = sum(p.numel() for p in model.parameters())
print(f'the total number of parameters in this model are {total_params}')

the total number of parameters in this model are 838


In [9]:
model.train()
for epoch in range(epochs):
    runnning_loss = 0 
    predicted_list = []
    labels_list = []
    print(f'this is epoch number {epoch}')
    for i, (input, label) in enumerate(stacking_training):
        optimiser.zero_grad()
        labels_list.append(label)
        label = torch.tensor(label).view(-1)
        if (i % 1000 == 0):
            print(f'this is the {i} iteration')
        outputs = model(input)
        label = F.one_hot(label, num_classes=2).float()
        loss = criterion(outputs, label)
        loss.backward()
        _, predicted = torch.max(outputs.data, 1)
        predicted_list.append(predicted.item())
        runnning_loss += loss.item()
        optimiser.step()
        if i % 1000 == 999:   
            print(' loss: %.3f accuracy: %.3f' %( runnning_loss / 10, 100 * accuracy_score(labels_list, predicted_list)))
            runnning_loss = 0

this is epoch number 0
this is the 0 iteration
 loss: 60.980 accuracy: 71.000
this is the 1000 iteration
 loss: 60.878 accuracy: 70.100
this is the 2000 iteration
this is epoch number 1
this is the 0 iteration
 loss: 54.342 accuracy: 71.000
this is the 1000 iteration
 loss: 51.932 accuracy: 70.050
this is the 2000 iteration
this is epoch number 2
this is the 0 iteration
 loss: 39.709 accuracy: 80.700
this is the 1000 iteration
 loss: 33.594 accuracy: 82.300
this is the 2000 iteration
this is epoch number 3
this is the 0 iteration
 loss: 30.106 accuracy: 86.200
this is the 1000 iteration
 loss: 28.594 accuracy: 86.600
this is the 2000 iteration
this is epoch number 4
this is the 0 iteration
 loss: 27.403 accuracy: 88.700
this is the 1000 iteration
 loss: 24.787 accuracy: 89.000
this is the 2000 iteration
this is epoch number 5
this is the 0 iteration
 loss: 24.570 accuracy: 89.800
this is the 1000 iteration
 loss: 21.554 accuracy: 90.400
this is the 2000 iteration
this is epoch number 6

In [10]:
model.eval()
loss = 0 
correct = 0
labels_list = []
predicted_list = []
total = 0
count = 0
with torch.no_grad():
    for i, (input, label) in enumerate(stacking_testing):
        label = torch.tensor(label)
        labels_list.append(label.item())
        label= label.view(-1)
        label = F.one_hot(label, num_classes=2).float()
        outputs = model(input)
        loss += criterion(outputs, label).item()
        predicted = outputs.argmax(dim = 1,keepdim = True)
        predicted_list.append(predicted.item())
    

In [11]:
print(f'the model accuracy in this case is {accuracy_score(labels_list, predicted_list)}')

the model accuracy in this case is 0.9523809523809523
